In [1]:
import pandas as pd
import numpy as np

import class_pig  

import glob
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import os
from scipy.optimize import curve_fit
"""
CUIDADO!!
"""
import warnings
warnings.filterwarnings("ignore")

In [2]:
from fitterClass import funcFitter
from utils import find_best_b, plot_exponentials

* d para Doente
* s para Saudavel

## Loading Data 

In [3]:
with open("data//doente.txt", "r") as file:
    d_data = file.readlines()
    #Cleaning
    d_data = np.array([list(map(float, row.split())) for row in d_data if row != '\n'])

with open("data//normal.txt", "r") as file:
    s_data = file.readlines()
    #Cleaning
    s_data = np.array([list(map(float, row.split())) for row in s_data if row != '\n'])
    
with open("data//k_func.txt", "r") as file:
    k_data = file.readlines()
    #Cleaning
    k_data = np.array([list(map(float, row.split())) for row in k_data if row != '\n'])

In [4]:
k_data

array([[  10.17812538, -197.85173035],
       [  20.55937481,  -50.39464188],
       [  15.21249962,  -95.13343811],
       [  25.51250076,   41.52325439],
       [  20.421875  ,   97.13526917],
       [  30.65312481,  271.86264038],
       [  25.34687519,  373.07644653],
       [  35.64375114,  563.88049316],
       [  30.46250057,  707.39672852],
       [  40.765625  ,  864.88140869]])

## Cleaning 

In [5]:
d_data[6,1] = 466 # valor corrigido

## Fitting Sigmoid

### Parameters

In [6]:
def sigmoid(x, a, b, c, d):
    return a + b/(1 + np.exp(-(x-c)/d))

def exponential(x, a, b, c, d):
    return a - b*np.exp(-(x-c)/d)

def exponential_cicle_fixed_b(x, a, c):
    return -a*np.exp(-best_b*x)+c

estimators = ["lm", "trf", "dogbox"]
funcs = [sigmoid, exponential]

### K data

In [ ]:
#Interpolando
fitter_k = funcFitter("mra26", "D", k_data[0::2, :], n_point = 5, estimators = estimators)
k_df = fitter_k.fit(funcs=funcs, interpolate=True, n_interp_point=100, interp_method="linear")
k_df.to_csv(os.path.join("data","k_df.csv"), index = False)
k_df.head(2)

In [ ]:
fitter_k.make_plot(df = k_df, n_best = 4)

In [ ]:
best_b, guess_zero = find_best_b(k_df, raw_pressures=k_data,  b_percentage_range=(0.05,0.125), step=0.001, plot_figure=False)
plot_exponentials(best_b, guess_zero, k_df, k_data, exponential_cicle_fixed_b)